# 选择维度作图

In [1]:
import pandas as pd
import numpy as np
import gc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from multiprocessing import Pool
import time
import pickle
import os
from sklearn.preprocessing import LabelEncoder

In [ ]:
import pandas as pd
import numpy as np
import gc
from multiprocessing import Pool
import time
import os
from sklearn.preprocessing import LabelEncoder
import networkx as nx
from node2vec import Node2Vec
import sys

his_trans = pd.read_csv('../data/historical_transactions.csv')
his_trans = his_trans[[ 'card_id','merchant_id' ]]
cacheRoot = '../cache/'
def create_edgelist(useData, dataID ,secNodeCol):
    le = LabelEncoder()
    datacp = useData[[dataID,secNodeCol ]]
    datacp = datacp[-datacp[secNodeCol].isnull()]
    datacp[secNodeCol] = le.fit_transform( datacp[secNodeCol] )+ 1000000
    each =datacp.groupby([dataID,secNodeCol])[dataID].agg({"trans_cnt":'count'}).reset_index()
    total = each.groupby(secNodeCol)["trans_cnt"].agg({ 'trans_cnt_total' :"sum"}).reset_index()
    gp = pd.merge(each,total,on=[ secNodeCol] )
    del datacp,each,total
    gp[ "weight"] = gp['trans_cnt']/gp['trans_cnt_total']
    gp = gp.drop(['trans_cnt','trans_cnt_total'],axis = 1)
    savename = cacheRoot +  'sourceEmb/{}_weighted_edglist.txt'.format(secNodeCol)
    np.savetxt(savename, gp.values, fmt=['%d','%d','%f'])
    return savename


def createEdgeFomat(fname):
    G = nx.Graph()
    f = open(fname,'r')
    lines = f.readlines()
    f.close()
    lines =[l.replace("\n","").split(" ")  for l in lines]
    lines = [[int(x[0]),int(x[1]),float(x[2])] for x in lines]
    edfname = fname.replace(".txt",".edgelist")
    
    for edg in lines:
        G.add_edge(edg[0], edg[1], weight=edg[2])
    print("\n-------------------------------------\n")
    print("saving fali name %s " % edfname)
    print("\n-------------------------------------\n")
    fh=open(edfname,'wb')
    nx.write_edgelist(G, fh)
    fh.close()
    return edfname


def emb_graph_2vec(inputpath,dim):
    print("input name will be ",inputpath)
    emb_name = inputpath.replace("weighted_edglist.edgelist","")
    print("emb_name will be ",emb_name)

    savename =inputpath.replace("weighted_edglist.edgelist",".emb")
    print("emb outfile name will be ",savename)
    if os.path.exists(savename):
        print("file alread exists in cache, please rename")
        sys.exit(1)

    graph = nx.read_edgelist(inputpath,create_using=nx.DiGraph())
    # Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
    node2vec = Node2Vec(graph, dimensions=dim, walk_length=30,p=0.3, q=1.3, num_walks=200, workers=10 ,weight_key='weight') 
    # Embed nodes
    print("training .... ")
    model = node2vec.fit(window=10, min_count=1, batch_words=4)
    print("training finished saving result... ")

    print("saving %s file to disk "%savename)
    # Save embeddings for later use
    model.wv.save_word2vec_format(savename)
    print("done")
savename = create_edgelist(his_trans,'card_id','merchant_id' )
edfname = createEdgeFomat(savename)
emb_graph_2vec(edfname,36)

In [1]:
import networkx as nx
from node2vec import Node2Vec

# Create a graph
graph = nx.fast_gnp_random_graph(n=100, p=0.5)

# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(graph, dimensions=64, walk_length=30,p=0.3, q=1.3, num_walks=200, workers=4,weight_key='weight') 

D:\Programfiles2\anaconda\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Computing transition probabilities: 100%|████████████████████████████████████████████| 100/100 [00:01<00:00, 84.76it/s]


In [2]:
dataroot = "../data/"
cacheRoot = "../cache/"


trainFiles  = {"transaction":'transaction_train_new.csv', "operation" :'operation_train_new.csv', "label" :'tag_train_new.csv'} 
validFiles = {"transaction":'transaction_round1_new.csv', "operation" :'operation_round1_new.csv'}
testFiles  = {"transaction":'test_transaction_round2.csv', "operation" :'test_operation_round2.csv'}


# train operation
train_oper = pd.read_csv( dataroot + trainFiles['operation'],encoding="utf8")
# train transaction
train_transac = pd.read_csv( dataroot + trainFiles['transaction'],encoding="utf8")
# train tag
# test operation
test_oper_r1 = pd.read_csv(dataroot + validFiles["operation"],encoding="utf8")
# test transaction
test_transac_r1 = pd.read_csv(dataroot + validFiles["transaction"],encoding="utf8")

test_oper_r2 = pd.read_csv(dataroot + testFiles["operation"],encoding="utf8")
# test transaction
test_transac_r2 = pd.read_csv(dataroot + testFiles["transaction"],encoding="utf8")




D:\Programfiles2\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
oper_tran_grpuse_col = ['UID',"device1","mac1","ip1","geo_code", "device_code1","device_code2","device_code3" ]
oper_col = [ 'UID','ip','ip_sub','wifi' ]# ip  ip1 fillna ip2  ip1_sub fillna ip2_sub
tran_col = ['UID', 'merchant', 'code2' ]

In [6]:

# ['ip1','mac1','mac2','geo_code']
# ['ip1','mac1','mac2' ,'merchant']
oper_tran_grpuse = pd.concat([ train_oper[oper_tran_grpuse_col],test_oper_r1[oper_tran_grpuse_col] , test_oper_r2[oper_tran_grpuse_col],train_transac[oper_tran_grpuse_col],test_transac_r1[oper_tran_grpuse_col],test_transac_r2[oper_tran_grpuse_col]])
train_oper['ip']   = train_oper['ip1'].fillna( train_oper['ip2'])
test_oper_r1['ip'] = test_oper_r1['ip1'].fillna( test_oper_r1['ip2'])
test_oper_r2['ip'] = test_oper_r2['ip1'].fillna( test_oper_r2['ip2'])

train_oper['ip_sub']   = train_oper['ip1_sub'].fillna( train_oper['ip2_sub'])
test_oper_r1['ip_sub'] = test_oper_r1['ip1_sub'].fillna( test_oper_r1['ip2_sub'])
test_oper_r2['ip_sub'] = test_oper_r2['ip1_sub'].fillna( test_oper_r2['ip2_sub'])
oper_use = pd.concat([ train_oper,test_oper_r1,test_oper_r2 ])
tran_use = pd.concat([ train_transac,test_transac_r1,test_transac_r2 ])


In [7]:


sourcedata = [{"usedata":oper_tran_grpuse,"useCol":oper_tran_grpuse_col},
              {"usedata":oper_use,"useCol":oper_col},
              {"usedata":tran_use,"useCol":tran_col}
             ]

def create_edgelist(useData, secNodeCol):
    le = LabelEncoder()
    datacp = useData[["UID",secNodeCol ]]
    datacp = datacp[-datacp[secNodeCol].isnull()]
    datacp[secNodeCol] = le.fit_transform( datacp[secNodeCol] )+ 1000000
    each =datacp.groupby(["UID",secNodeCol])['UID'].agg({"trans_cnt":'count'}).reset_index()
    total = each.groupby(secNodeCol)["trans_cnt"].agg({ 'trans_cnt_total' :"sum"}).reset_index()
    gp = pd.merge(each,total,on=[ secNodeCol] )
    del datacp,each,total
    gp[ "ratio"] = gp['trans_cnt']/gp['trans_cnt_total']
    gp = gp.drop(['trans_cnt','trans_cnt_total'],axis = 1)
    savename = cacheRoot +  'sourceEmb/{}_weighted_edglist_filytypeTxt.txt'.format(secNodeCol)
    np.savetxt(savename, gp.values, fmt=['%d','%d','%f'])
    gp = gp.drop("ratio",axis = 1)
    savenameForDeepWalk = cacheRoot +  '{}_weighted_edglist_DeepWalk.txt'.format(secNodeCol)
    np.savetxt(savenameForDeepWalk, gp.values, fmt=['%d','%d'])
    del gp



for spec in sourcedata:
    for c in spec["useCol"]:
        if c != "UID":
            create_edgelist(spec["usedata"], c)
            gc.collect()

D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:11: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
D:\Programfiles2\anaconda\lib\site-packages\ipykernel\__main__.py:12: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# dict(zip(df['keys'],df['vals'] ))
df = pd.DataFrame()
keys = ['a','b','c','a']
vals = [1,2,3,4]
df['keys'] = keys
df['vals'] = vals

In [18]:
le = LabelEncoder()
dict(zip(keys,le.fit_transform( keys )))

{'a': 0, 'b': 1, 'c': 2}

In [10]:
dict(zip(df['keys'],df['vals'] ))

{'a': 1, 'b': 2, 'c': 3}

In [4]:
import networkx as nx
def createEdgeFomat(fname):
    G = nx.Graph()
    f = open(fname,'r')
    lines = f.readlines()
    f.close()
    lines =[l.replace("\n","").split(" ")  for l in lines]
    lines = [[int(x[0]),int(x[1]),float(x[2])] for x in lines]
    edfname = fname.replace(".txt",".edgelist")
    
    for edg in lines:
        G.add_edge(edg[0], edg[1], weight=edg[2])
    print("\n-------------------------------------\n")
    print("saving fali name %s " % edfname)
    print("\n-------------------------------------\n")
    fh=open(edfname,'wb')
    nx.write_edgelist(G, fh)
    fh.close()

for f in os.listdir(cacheRoot):
    if "DeepWalk" not in f:
        print("creating %s edge format for node2vec embedding ... " % (f.replace( "_edglist_filytypeTxt.txt", "" )) )
        createEdgeFomat(cacheRoot + f)
        print(f.split(".")[0],"finish")


creating code2_weighted edge format for node2vec embedding ... 

-------------------------------------

saving fali name ./compFeatures/sourceEmb/code2_weighted_edglist_filytypeTxt.edgelist 

-------------------------------------

code2_weighted_edglist_filytypeTxt finish
creating device1_weighted edge format for node2vec embedding ... 

-------------------------------------

saving fali name ./compFeatures/sourceEmb/device1_weighted_edglist_filytypeTxt.edgelist 

-------------------------------------

device1_weighted_edglist_filytypeTxt finish
creating device_code1_weighted edge format for node2vec embedding ... 

-------------------------------------

saving fali name ./compFeatures/sourceEmb/device_code1_weighted_edglist_filytypeTxt.edgelist 

-------------------------------------

device_code1_weighted_edglist_filytypeTxt finish
creating device_code2_weighted edge format for node2vec embedding ... 

-------------------------------------

saving fali name ./compFeatures/sourceEmb/d